In [44]:
from sqlalchemy import Column, Integer, Numeric, String, Date, Index, Text, text, Float, create_engine  # The architects of our database schema, crafting the structure of our data haven.
from sqlalchemy.orm import sessionmaker, declarative_base  # Craftsmen who build the bridge between our Pythonic realm and the database underworld.
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import declarative_base, sessionmaker

from rapidframes import RapiDF, QueryBuilder

import threading
import warnings
from typing import List, Dict
import polars as pl
import pandas as pd

ModuleNotFoundError: No module named 'rapidframes'

In [2]:
!pip3 install -r requirements.txt 

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import sys
import os

sys.path.append('/home/ubuntu/OTB/rapidframes/')
sys.path.append('/home/ubuntu/OTB/BMAPS/routes')

In [5]:
import datetime
from collections import namedtuple
from datetime import datetime, timedelta, date
from time import ctime
from typing import Dict, List
import pandas as pd
import numpy as np
import polars as pl
from rapidframes import QueryBuilder, RapiDF
from warnings import simplefilter

In [6]:
STORAGE_URL = 'lmdb://databases/bmaps_data?map_size=50GB'
rf = RapiDF(STORAGE_URL)

In [7]:
# Library RA_DATA got created
rf.get_library("RA_DATA").list_symbols()

['ra_item_joined',
 'kpi_trnx',
 'item_master',
 'ra_kpi_joined',
 'stock_trnx',
 'ra_stock_joined',
 'cst_trnx',
 'sales_filtered',
 'filtered_symobl',
 'ra_trnx',
 'ra_sales_join']

In [8]:
LIBRARY_NAME = "RA_DATA"

In [45]:
class PostgresDatabaseSecretsHolder:
    # Configuring the warehouse to store the contraband.
    USERNAME = "postgres"
    PASSWORD = "6eA1Wyp6l'sj"
    HOST     = "localhost"
    PORT     = "5432"
    DATABASE = "bmaps"

# Establish a connection to the MariaDB and Postgre Server, like making an untraceable phone call to an old friend.
# Connecting to the MariaDB and Postgre Instances is akin to reaching out to a contact in a foreign land.
# The connection stringv, a secret formula for establishing a clandestine rendezvous.
# mariadb_connection_string = f"mysql+pymysql://{MariaDBDatabaseSecretsHolder.USERNAME}:{MariaDBDatabaseSecretsHolder.6eA1Wyp6l'sj}\
# @{MariaDBDatabaseSecretsHolder.HOST}:{MariaDBDatabaseSecretsHolder.PORT}/{MariaDBDatabaseSecretsHolder.DATABASE}"

postgres_connection_string = f"postgresql+psycopg2://{PostgresDatabaseSecretsHolder.USERNAME}:{PostgresDatabaseSecretsHolder.PASSWORD}\
@{PostgresDatabaseSecretsHolder.HOST}:{PostgresDatabaseSecretsHolder.PORT}/{PostgresDatabaseSecretsHolder.DATABASE}"

# Create the engines, the heart of our operation, silent yet powerful.
# mariadb_engine = create_engine(mariadb_connection_string)
postgres_engine = create_engine(postgres_connection_string)
# The blueprint of our data structure, meticulous and precise.
Base = declarative_base()
Session = sessionmaker(bind=postgres_engine)
session = Session()

class SalesDataTrnx(Base):
    __tablename__ = 'salesdata_trnx'
    ID               = Column(Integer, primary_key=True, autoincrement=True)
    INVENTLOCATIONID = Column(String(25))
    ITEMID           = Column(String(50))
    SALESQTY         = Column(Numeric(15, 3))
    LINEDISC         = Column(Numeric(15, 3))
    LINEAMOUNT       = Column(Numeric(15, 3))
    SALESPRICE       = Column(Numeric(15, 3))
    COSTPRICE        = Column(Numeric(15, 3))
    INVOICEDATE      = Column(Date())
    channel          = Column(String(50))
    family           = Column(String(30))

    __table_args__ = (
        Index('salesdata_trnx_INVOICEDATE_IDX', 'INVOICEDATE'),
        Index('salesdata_trnx_ITEMID_IDX', 'ITEMID', 'INVENTLOCATIONID', 'channel', 'family'),
        Index('salesdata_trnx_LINEAMOUNT_IDX', 'LINEAMOUNT'),
        Index('salesdata_trnx_INVENTLOCATIONID_1_IDX', 'INVENTLOCATIONID', 'ITEMID', 'channel'),
        Index('salesdata_trnx_INVENTLOCATIONID_2_IDX', 'INVENTLOCATIONID', 'channel', 'INVOICEDATE', 'ITEMID', 'family'),
        Index('salesdata_trnx_family_IDX', 'family', 'ITEMID', 'LINEAMOUNT'),
        Index('salesdata_trnx_family_amt_IDX', 'family', 'LINEAMOUNT'),
    )

class RATransactionTable(Base):
    __tablename__ = 'ra_transaction_table'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Channel = Column(Text)
    country = Column(Text)
    Region = Column(Text)
    area = Column(Text)
    city = Column(Text)
    StoreId = Column(Integer)
    Store = Column(Text)
    season = Column(Text)
    Department = Column(Text)
    Family = Column(Text)
    SubFamily = Column(Text)
    Category = Column(Text)
    DOM_COMM = Column(Text)
    SubCategory = Column(Text)
    ExtendedSubCategory = Column(Text)
    Supplier = Column(Text)
    SubCategorySupplier = Column(Text)
    AssemblyCodeNickName = Column(Text)
    Status = Column(Text)
    ENDOFLife = Column(Text)
    Description = Column(Text)
    ItemLookupCode = Column(Text)
    BudgetYear = Column(Float)
    BDate = Column(Date)
    BudgetAmount = Column(Float)
    UnitsBuyBySku = Column(Numeric)
    BudgetGrossMarginP = Column(Float)
    BudgetCost = Column(Float)
    BudgetQTY = Column(Numeric)
    OpeningStock = Column(Numeric)
    ClosingStock = Column(Numeric)
    StockReceivedQty = Column(Integer)
    TotalPurchaseQty = Column(Numeric)
    StockOnHandQty = Column(Numeric)
    CurrentStockCost = Column(Float)
    budget_quarter = Column(Text)
    budget_month = Column(Text)
    budget_week = Column(Text)
    budget_day = Column(Text)

In [46]:
import logging
import psycopg2
from psycopg2 import OperationalError
import traceback

# Enable logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

try:
    conn = psycopg2.connect(dbname="bmaps", user="mohit", host = "localhost", port = "5432", password = "password")
    conn.close()
    print(True)
except Exception as e:
    print(traceback.format_exc())
    print(e)

True


In [12]:
# When no fetch
lib = rf.get_library('RA_DATA', create_if_missing=True)

In [13]:
def fetch_item_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)
    print(f"[INFO] FETCH ITEM_MASTER TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    item_data    = ps_conn.execute(text('''SELECT "ItemLookupCode" AS "ITEMID",
                                                 MAX("Description") AS "Description",
                                                 MAX("Department") AS "Department",
                                                 MAX("CategoryName") AS "Category",
                                                 MAX("Family") AS "Family",
                                                 MAX("subFamily") AS "SubFamily",
                                                 MAX("SubCategory") AS "SubCategory",
                                                 MAX("ExtendedSubCategory") AS "ExtendedSubCategory",
                                                 MAX("SubCategorySupplier") AS "SubCategorySupplier",
                                                 MAX("AssemblyCodeNickname") AS "AssemblyCodeNickName",
                                                 MAX("ENDOFLife") AS "ENDOFLife",
                                                 MAX("DOM_COMM") AS "DOM_COMM",
                                                 MAX("Status") AS "Status",
                                                 MAX("Supplier") AS "Supplier"
                                                 FROM 
                                                    itemmaster
                                                 GROUP BY 
                                                    "ItemLookupCode"'''))
    item_master      = pd.DataFrame(item_data, columns=item_data.keys())
    item_master["ENDOFLife"]  = item_master["ENDOFLife"].fillna("")
    item_master['ITEMID'] = item_master['ITEMID'].astype(str)

    lib.write("item_master", item_master, prune_previous_versions=True) # , prune_previous_versions=True
    return lib
lib = fetch_item_data()

[INFO] FETCH ITEM_MASTER TRANSACTIONS


In [15]:
q = QueryBuilder()
item_master = pl.DataFrame(rf.get_library("RA_DATA").read('item_master', query_builder=q).data)
type(item_master.head(2))

polars.dataframe.frame.DataFrame

In [16]:
def get_type_name(val):
    return type(val).__name__

    # Apply the function to the column
# item_master = item_master.with_columns(pl.col('ITEMID').apply(get_type_name).alias('type_name'))

    # Count the values
    # item_master = item_master.groupby('type_name').agg(pl.count('type_name'))

In [14]:
# item_master['type_name'].value_counts()

In [17]:
def fetch_ra_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Fetch RA Transactions
    print(f"[INFO] FETCH RA TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    ra_data    = ps_conn.execute(text("SELECT * from ra_table; "))
    ra_df      = pd.DataFrame(ra_data, columns=ra_data.keys())
    ra_df['ITEMID'] = ra_df['ITEMID'].astype(str)
    ra_df["Budget_date"]      = pd.to_datetime(ra_df["Budget_date"])
    # ra_df["ENDOFLife"]  = ra_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # ra_df[cols_to_conv] = ra_df[cols_to_conv].apply(pd.to_numeric)

    lib.write("ra_trnx", ra_df, prune_previous_versions=True)
    ra_df = None

    return lib

lib = fetch_ra_data()

[INFO] FETCH RA TRANSACTIONS


In [18]:
q = QueryBuilder()
ra_trnx = pl.DataFrame(rf.get_library("RA_DATA").read('ra_trnx', query_builder=q).data)

In [19]:
ra_trnx['Budget_date'].value_counts()
max(ra_trnx['Budget_date'])

datetime.datetime(2024, 4, 20, 0, 0)

In [20]:
ra_trnx.shape

(9792848, 11)

In [21]:
# ra_trnx = ra_trnx.with_columns(pl.col('ITEMID').apply(get_type_name).alias('type_name'))
# ra_trnx['type_name'].value_counts()

In [22]:
ra_trnx['Budget_date'].value_counts()

Budget_date,count
datetime[ns],u32
2024-03-27 00:00:00,4896424
2024-04-20 00:00:00,4896424


In [23]:
def fetch_cst_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Fetch CST Transactions
    print(f"[INFO] FETCH CURRENT SALES TRANSACTIONS")
    
    ps_session            = sessionmaker(bind=postgres_engine)
    ps_conn               = ps_session()
    cst_data              = ps_conn.execute(text("SELECT * from current_sales_trnx;"))
    cst_df                = pd.DataFrame(cst_data, columns=cst_data.keys())
    cst_df["INVOICEDATE"] = pd.to_datetime(cst_df["INVOICEDATE"])
    cols_to_conv          = ["LINEAMOUNT", "SALESQTY", "COSTPRICE", "gross_sales", "historical_year"]
    cst_df[cols_to_conv]  = cst_df[cols_to_conv].apply(pd.to_numeric)
    
    lib.write("cst_trnx", cst_df, prune_previous_versions=True) # , prune_previous_versions=True

    return lib


lib = fetch_cst_data()

[INFO] FETCH CURRENT SALES TRANSACTIONS


In [24]:
q = QueryBuilder()
cst_trnx  = pl.DataFrame(rf.get_library("RA_DATA").read('cst_trnx', query_builder=q).data)

In [25]:
cst_trnx.describe()


statistic,ITEMID,channel,INVENTLOCATIONID,LINEAMOUNT,SALESQTY,COSTPRICE,gross_sales,historical_year,INVOICEDATE,history_quarter,history_month,history_week,history_day
str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str
"""count""","""7570000""","""7550913""","""7570001""",7.569999e6,7.569999e6,7.569999e6,7.569999e6,7.569999e6,"""7569999""","""7569999""","""7569999""","""7569999""","""7569999"""
"""null_count""","""1""","""19088""","""0""",2.0,2.0,2.0,2.0,2.0,"""2""","""2""","""2""","""2""","""2"""
"""mean""",null,null,null,6688.88592,14.731555,4613.288953,6805.761984,2022.056929,"""2022-07-14 13:…",null,null,null,null
"""std""",null,null,null,75202.841194,461.447365,37146.798065,75826.438706,0.871128,null,null,null,null,null
"""min""","""1AAMACWLAMI010…","""""","""""",-8.7073e6,-263157.0,0.0,-8.7073e6,2021.0,"""2021-01-01 00:…","""1""","""April ""","""1""","""Friday """
"""25%""",null,null,null,320.0,1.0,285.5,330.0,2021.0,"""2021-11-06 00:…",null,null,null,null
"""50%""",null,null,null,2049.0,1.0,1738.75,2090.0,2022.0,"""2022-06-28 00:…",null,null,null,null
"""75%""",null,null,null,6760.0,1.0,5292.88,6839.0,2023.0,"""2023-03-28 00:…",null,null,null,null
"""max""","""transferfeeson…","""WAREHOUSE""","""re-9014""",9.0877e7,524899.0,2.9371e7,9.0877e7,2024.0,"""2024-02-27 00:…","""4""","""September""","""9""","""Wednesday"""


In [26]:
lib

Library(s(config=LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)), path=RA_DATA, storage=lmdb_storage)

In [27]:
cst_trnx['history_day'].value_counts()

history_day,count
str,u32
"""Tuesday """,1099861
"""Saturday """,1086948
null,2
"""Wednesday""",1131032
"""Sunday """,1108307
"""Thursday """,1170404
"""Friday """,851371
"""Monday """,1122076


In [28]:
def fetch_stock_data():
  lib = rf.get_library('RA_DATA', create_if_missing=True)

  # Get the opening stock
  print(f"[INFO] FETCH STOCK TRANSACTIONS")

  ps_session = sessionmaker(bind=postgres_engine)
  ps_conn    = ps_session()

  stock_data    = ps_conn.execute(text('''	SELECT "ItemLookupCode" AS "ITEMID" ,
                                          "StoreID" AS "INVENTLOCATIONID",
                                          SUM("Quantity") AS "stock_on_hand_qty",
                                          SUM("AvailableQTY") AS "opening_stock",
                                          SUM("AvailableQTY") AS "closing_stock",
                                          AVG("Price") AS "current_stock_cost_at_retail",
                                          0 AS "stock_received_qty",
                                          SUM("Cost") AS "opening_stock_at_cost",
                                          0 AS "adjustment_at_cost",
                                          0 AS "adjustment_at_retail",
                                          MAX("SnapShotTime") AS "stock_date"
                                          FROM stock 
                                          GROUP BY "ItemLookupCode","StoreID";
                                    '''
                                  ))
  stock_df      = pd.DataFrame(stock_data, columns=stock_data.keys())
  cols_to_conv        = ["stock_on_hand_qty", "opening_stock", "closing_stock", "current_stock_cost_at_retail"]
  stock_df[cols_to_conv] = stock_df[cols_to_conv].apply(pd.to_numeric)
  # problematic_rows = stock_df[pd.to_datetime(stock_df["stock_date"], errors='coerce').isna()]
  # print(problematic_rows, 'rows causing datetime err')
  # print('hiiii')
  stock_df["stock_date"] = pd.to_datetime(stock_df["stock_date"], errors='coerce')

  lib.write("stock_trnx", stock_df, prune_previous_versions=True)
  # stock_df = None
  return lib
lib = fetch_stock_data()

[INFO] FETCH STOCK TRANSACTIONS


In [ ]:
# type(stock_df['stock_date'].dtypes)

In [29]:
q = QueryBuilder()
stock_trnx = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('stock_trnx', query_builder=q).data)

In [30]:
stock_trnx['stock_date'].value_counts()

stock_date,count
datetime[ns],u32
2024-02-26 00:00:00,112012


In [31]:
maximum_stock_available_date = datetime.date(stock_trnx['stock_date'].dt.max())
today_is = date.today()
print(f"INFO OF CLOSING IS AVAILABLE TILL :{maximum_stock_available_date} AND CURRENT DATE IS: {today_is}")

INFO OF CLOSING IS AVAILABLE TILL :2024-02-26 AND CURRENT DATE IS: 2024-04-23


In [32]:
if maximum_stock_available_date < date.today(): 
    cst_trnx = cst_trnx.with_columns((pl.when(cst_trnx['INVOICEDATE'].cast(pl.Date) > maximum_stock_available_date, cst_trnx['INVOICEDATE'].cast(pl.Date) < today_is)
                                                    .then(cst_trnx['SALESQTY'])
                                                    .otherwise(0)).alias('to_calculate_the_stock_till_now'))

In [ ]:
# sales_df['INVOICEDATE'].isnull().sum()   

In [ ]:
# sales_df = cst_trnx.to_pandas()

In [ ]:
# sales_df.head()
# lib.write("cst_trnx", sales_df, prune_previous_versions=True)

In [ ]:
# stock_trnx.shape

In [33]:
type(stock_trnx['stock_date'][0])
stock_trnx['stock_date'][0]
datetime.date(stock_trnx['stock_date'][0])
stock_trnx['stock_date'].dt.max()

datetime.datetime(2024, 2, 26, 0, 0)

In [34]:
stock_trnx.shape

(112012, 11)

In [35]:
def fetch_kpi_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Get the opening stock
    print(f"[INFO] FETCH KPI TRANSACTIONS")

    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()

    kpi_data    = ps_conn.execute(text("SELECT * from kpi_table;"))
    kpi_df      = pd.DataFrame(kpi_data, columns=kpi_data.keys())

    # ra_df["Budget_date"]      = pd.to_datetime(ra_df["Budget_date"])
    # ra_df["ENDOFLife"]  = ra_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["budget_qty"]

    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # ra_df[cols_to_conv] = ra_df[cols_to_conv].apply(pd.to_numeric)
  
    lib.write("kpi_trnx", kpi_df, prune_previous_versions=True)#,prune_previous_versions=True
    return lib
lib = fetch_kpi_data()


[INFO] FETCH KPI TRANSACTIONS


In [36]:
q = QueryBuilder()
kpi_trnx  = pl.DataFrame(rf.get_library("RA_DATA").read("kpi_trnx", query_builder=q).data)

In [37]:
kpi_trnx.shape

(27978, 15)

In [38]:
rf.list_libraries()

['RA_DATA', 'SP_DATA']

In [39]:
print(*map(lambda item:(print(item), item)[1], lib.list_versions().items()), 'the symbol version')

(stock_trnx_v10, (date=2024-04-23 05:15:36.894904309+00:00))
(sales_filtered_v10, (date=2024-04-23 05:11:14.824749173+00:00))
(ra_trnx_v10, (date=2024-04-23 05:13:47.848772330+00:00))
(ra_stock_joined_v4, (date=2024-04-19 08:51:00.951067239+00:00))
(ra_sales_join_v10, (date=2024-04-23 05:11:14.846339113+00:00))
(ra_sales_join_v9, (date=2024-04-23 04:24:54.272247054+00:00))
(ra_sales_join_v8, (date=2024-04-22 06:03:28.468502119+00:00))
(ra_sales_join_v7, (date=2024-04-22 03:57:23.037962654+00:00))
(ra_sales_join_v6, (date=2024-04-21 05:26:41.660109842+00:00))
(ra_sales_join_v5, (date=2024-04-20 13:59:22.122174493+00:00))
(ra_sales_join_v4, (date=2024-04-20 07:39:49.470835400+00:00))
(ra_sales_join_v3, (date=2024-04-20 07:37:21.894614255+00:00))
(ra_sales_join_v2, (date=2024-04-19 06:10:09.011982477+00:00))
(ra_sales_join_v1, (date=2024-04-19 06:09:30.434916325+00:00))
(ra_sales_join_v0, (date=2024-04-19 05:58:49.030875865+00:00))
(ra_kpi_joined_v2, (date=2024-04-19 05:57:02.941677834+00

In [40]:
import rapidframes
class rapidframe_symbols:
    #Class attributes
    ITEM_MASTER  = 'item_master'
    RA_TRNX      = 'ra_trnx'
    CST_TRNX     = 'cst_trnx'
    STOCK_TRNX   = 'stock_trnx'
    KPI_TRNX     = 'kpi_trnx'
    # Join the symbols
    ITEM_RA      = 'item_ra_joined'
    RA_STOCK     = 'ra_stock_joined'
    STOCK_CST    = 'stock_cst_joined'
    CST_KPI      = 'cst_kpi_joined'
    
    def combine_the_rapid_frame_symbols(self, rf : rapidframes.repidf.RapiDF, Stack : str, First, Last, LIBRARY_NAME, join_element):
        
        rf.get_library(LIBRARY_NAME).write(
            Stack,
            First.join(Last,
            on=join_element,
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )
# symbols = rapidframe_symbols()

In [ ]:
# symbols.combine_the_rapid_frame_symbols(rf, symbols.ITEM_RA, item_master, ra_trnx, LIBRARY_NAME, list('ITEMID'))

In [41]:
ra_trnx

Store,area,Region,adjusted_budget_gross_margin_percent,budget_amount,budget_cost,budget_qty,ITEMID,Channel,INVENTLOCATIONID,Budget_date
str,str,str,f64,f64,f64,f64,str,str,str,datetime[ns]
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOCMEF24NW…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOCMFE24AN…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOCMFE85NW…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4H5…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4JO…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4S5…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4SF…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4SG…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00
"""BTECHX Gehan2 …","""Tanash""","""CAIRO""",20.0,0.0,-0.0,0.0,"""1BSOCOGMPLA4SL…","""B.TECH X""","""Re-1508""",2024-03-27 00:00:00


In [42]:
rf.get_library("RA_DATA").write(
            'ra_item_joined',
            ra_trnx.join(item_master,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_item_joined', library='RA_DATA', data=n/a, version=9, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [43]:
q=QueryBuilder()
ra_item_joined = pl.DataFrame(rf.get_library("RA_DATA").read('ra_item_joined', query_builder=q).data)

In [44]:
ra_item_joined.shape

(9792848, 24)

In [56]:
q=QueryBuilder()
stock_trnx = pl.DataFrame(rf.get_library("RA_DATA").read('ra_item_joined', query_builder=q).data)
stock_trnx.columns

['ITEMID',
 'INVENTLOCATIONID',
 'stock_on_hand_qty',
 'opening_stock',
 'closing_stock',
 'current_stock_cost_at_retail',
 'stock_received_qty',
 'opening_stock_at_cost',
 'adjustment_at_cost',
 'adjustment_at_retail',
 'stock_date']

In [46]:
rf.get_library("RA_DATA").write(
            'ra_stock_joined',
            ra_item_joined.join(stock_trnx,
            on=['ITEMID', 'INVENTLOCATIONID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_stock_joined', library='RA_DATA', data=n/a, version=5, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [55]:
q=QueryBuilder()
ra_stock_joined = pl.DataFrame(rf.get_library("RA_DATA").read('ra_stock_joined', query_builder=q).data)
ra_stock_joined.columns

['Store',
 'area',
 'Region',
 'adjusted_budget_gross_margin_percent',
 'budget_amount',
 'budget_cost',
 'budget_qty',
 'ITEMID',
 'Channel',
 'INVENTLOCATIONID',
 'Budget_date',
 'Description',
 'Department',
 'Category',
 'Family',
 'SubFamily',
 'SubCategory',
 'ExtendedSubCategory',
 'SubCategorySupplier',
 'AssemblyCodeNickName',
 'ENDOFLife',
 'DOM_COMM',
 'Status',
 'Supplier',
 'Store_right',
 'area_right',
 'Region_right',
 'adjusted_budget_gross_margin_percent_right',
 'budget_amount_right',
 'budget_cost_right',
 'budget_qty_right',
 'Channel_right',
 'Budget_date_right',
 'Description_right',
 'Department_right',
 'Category_right',
 'Family_right',
 'SubFamily_right',
 'SubCategory_right',
 'ExtendedSubCategory_right',
 'SubCategorySupplier_right',
 'AssemblyCodeNickName_right',
 'ENDOFLife_right',
 'DOM_COMM_right',
 'Status_right',
 'Supplier_right']

In [48]:
stock_trnx['stock_date'].value_counts()

ColumnNotFoundError: stock_date

In [ ]:
ra_stock_joined = ra_stock_joined.with_columns(pl.col('stock_date').fill_null(datetime(2024,2,26,00,00,00)))

In [ ]:
cst_trnx['INVOICEDATE'].cast(pl.Date)[0]
cst_trnx['INVOICEDATE'].dt.min()

datetime.datetime(2021, 1, 1, 0, 0)

In [ ]:
cst_trnx['historical_year']

historical_year
f64
2022.0
2022.0
2022.0
2022.0
2022.0
2022.0
2022.0
2022.0
2022.0


In [ ]:
cst_trnx.melt(variable_name="column").group_by(pl.all()).len()['column'].to_list()

['gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'gross_sales',
 'COSTPRICE',
 'COSTPRICE',
 'gross_sales',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'LINEAMOUNT',
 'gross_sales',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'gross_sales',
 'gross_sales',
 'gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'COSTPRICE',
 'gross_sales',
 'COSTPRICE',
 'gross_sales',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'gross_sales',
 'gross_sales',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'COSTPRICE',
 'COSTPRICE',
 'COSTPRICE',
 'gross_sale

In [49]:
q=QueryBuilder()
kpi_trnx = pl.DataFrame(rf.get_library("RA_DATA").read('kpi_trnx', query_builder=q).data)

In [50]:
rf.get_library("RA_DATA").write(
            'ra_kpi_joined',
            ra_stock_joined.join(kpi_trnx,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=3, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [51]:
q=QueryBuilder()
ra_kpi_joined  = pl.DataFrame(rf.get_library("RA_DATA").read('ra_kpi_joined', query_builder=q).data)

In [52]:
ra_kpi_joined.shape

(19594944, 60)

In [53]:
ra_kpi_joined['stock_date'].value_counts()

ColumnNotFoundError: stock_date

In [54]:
fill_date = ra_kpi_joined['stock_date'].dt.max()
ra_kpi_joined = ra_kpi_joined.with_columns(stock_date = pl.col('stock_date').fill_null(fill_date))
lib.write('ra_kpi_joined', ra_kpi_joined.to_pandas(), prune_previous_versions=True)

ColumnNotFoundError: stock_date

In [59]:
print(*map(lambda item:(print(item), item)[1], dict(zip(ra_kpi_joined.columns, ra_kpi_joined.dtypes)).items()), 'the symbol version')

('Store', String)
('area', String)
('Region', String)
('adjusted_budget_gross_margin_percent', Float64)
('budget_amount', Float64)
('budget_cost', Float64)
('budget_qty', Float64)
('ITEMID', String)
('Channel', String)
('INVENTLOCATIONID', String)
('Budget_date', Datetime(time_unit='ns', time_zone=None))
('Description', String)
('Department', String)
('Category', String)
('Family', String)
('SubFamily', String)
('SubCategory', String)
('ExtendedSubCategory', String)
('SubCategorySupplier', String)
('AssemblyCodeNickName', String)
('ENDOFLife', String)
('DOM_COMM', String)
('Status', String)
('Supplier', String)
('stock_on_hand_qty', Float64)
('opening_stock', Float64)
('closing_stock', Float64)
('current_stock_cost_at_retail', Float64)
('stock_received_qty', Float64)
('opening_stock_at_cost', Float64)
('adjustment_at_cost', Float64)
('adjustment_at_retail', Float64)
('stock_date', Datetime(time_unit='us', time_zone=None))
('article_score_sale', Float64)
('article_score_abc', Float64)
(

In [60]:
def filter_symbol(rf: RapiDF, library_name: str, symbol_name: str, columns: List[str]=None, filter_set: Dict[str, str | int | list]= {}, return_as_pl: bool=True):
    # Initialize the QueryBuilder
    q = QueryBuilder()

    # Apply filters dynamically
    for column, conditions in filter_set.items():
        for operator, value in conditions.items():
            if operator == "_gt_": # Greater Than
                q = q[(q[column] > value)]
            elif operator == "_gte_": # Greater than or Equal To
                q = q[(q[column] >= pd.to_datetime(value))]
            elif operator == "_lt_": # Less Than
                q = q[(q[column] < value)]
            elif operator == "_lte_": # Less Than or Equal To
                q = q[(q[column] <= pd.to_datetime(value))]
            elif operator == "_eq_": # Equal To
                q = q[(q[column] == value)]
            elif operator == "_isin_": # Is in the list
                q = q[(q[column].isin(value))]

    filtered_df = rf.get_library(library_name).read(symbol_name, columns=columns, query_builder=q).data
    return pl.DataFrame(filtered_df) if return_as_pl else filtered_df


In [61]:
history_start_date_str  = "2021-01-01"
history_end_date_str    = "2023-12-16"

forecast_start_date_str = "2024-01-01"
forecast_end_date_str   = "2024-12-31"


In [73]:
list(ra_kpi_joined.columns)

['Store',
 'area',
 'Region',
 'adjusted_budget_gross_margin_percent',
 'budget_amount',
 'budget_cost',
 'budget_qty',
 'ITEMID',
 'Channel',
 'INVENTLOCATIONID',
 'Budget_date',
 'Description',
 'Department',
 'Category',
 'Family',
 'SubFamily',
 'SubCategory',
 'ExtendedSubCategory',
 'SubCategorySupplier',
 'AssemblyCodeNickName',
 'ENDOFLife',
 'DOM_COMM',
 'Status',
 'Supplier',
 'stock_on_hand_qty',
 'opening_stock',
 'closing_stock',
 'current_stock_cost_at_retail',
 'stock_received_qty',
 'opening_stock_at_cost',
 'adjustment_at_cost',
 'adjustment_at_retail',
 'stock_date',
 'article_score_sale',
 'article_score_abc',
 'article_score_ae',
 'article_score_speed',
 'article_score_terminal',
 'article_score_margin',
 'article_score_sell',
 'article_score_markdown',
 'article_score_core',
 'article_score_quartile',
 'article_score_sortimeter',
 'btech_vs_sortimeter',
 'store_sku_count',
 'price']

In [63]:
ra_kpi_joined['Budget_date'].value_counts()

Budget_date,count
datetime[ns],u32
2024-03-27 00:00:00,4896424
2024-03-28 00:00:00,4896424


In [143]:
ra_kpi_joined['Budget_date'][0]

datetime.datetime(2024, 3, 27, 0, 0)

In [144]:
ra_kpi_joined = ra_kpi_joined.fill_null(0)

In [145]:
lib.append('ra_kpi_joined', ra_kpi_joined.to_pandas(), prune_previous_versions=True)

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=2, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [79]:
%reset -f

In [2]:
pip install asyncpg

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncpg

In [2]:
conn = await asyncpg.connect(
        user="mohit",
        password="password",
        database="bmaps",
        host="localhost", # 43.204.167.201
        port=5433,
    )

In [3]:
conn.execute("CALL get_sp_otp();")

<coroutine object Connection.execute at 0x79ab1441d9a0>

In [4]:
query = """
SELECT "ITEMID", "Description", "Department", "Category", "Family", "SubFamily", "SubCategory",
        "ExtendedSubCategory", "SubCategorySupplier", "AssemblyCodeNickName", "ENDOFLife",
        "DOM_COMM", "Status", "Supplier", "Store", "area", "Region",
        "adjusted_budget_gross_margin_percent", "budget_amount", "budget_cost", "budget_qty",
        "Channel", "INVENTLOCATIONID", "Budget_date", "stock_on_hand_qty", "opening_stock",
        "closing_stock", "current_stock_cost_at_retail", "stock_received_qty",
        "opening_stock_at_cost", "adjustment_at_cost", "adjustment_at_retail", "stock_date",
        "article_score_sale", "article_score_abc", "article_score_ae", "article_score_speed",
        "article_score_terminal", "article_score_margin", "article_score_sell",
        "article_score_markdown", "article_score_core", "article_score_quartile",
        "article_score_sortimeter", "btech_vs_sortimeter", "store_sku_count", "price"
FROM ra_item_stock_kpi_temp;
"""

In [5]:
# Fetch results from temp_result table
records = await conn.fetch(query)

In [10]:
from collections import OrderedDict

In [11]:
def record_to_dict(record):
    return OrderedDict(record)

In [19]:
Rec_to_dict = lambda record : OrderedDict(record)

In [20]:
Rec_to_dict(records[0])

OrderedDict([('ITEMID', '1TSATVSM43T53000L007'),
             ('Description', 'TV SAMSUNG 43T5300 FHD SMART'),
             ('Department', 'DOMESTIC SAMSUNG'),
             ('Category', 'TV'),
             ('Family', 'TV'),
             ('SubFamily', 'TV DOM'),
             ('SubCategory', 'TV-43-SMRT'),
             ('ExtendedSubCategory', 'TV-43-SMRT'),
             ('SubCategorySupplier', 'TV-43-SMRT-SAMSUNG'),
             ('AssemblyCodeNickName',
              '1TSATVSM43T53000L007\nTV SAMSUNG 43T5300 FHD SMART'),
             ('ENDOFLife', None),
             ('DOM_COMM', 'DOM'),
             ('Status', 'No'),
             ('Supplier', 'SAMSUNG'),
             ('Store', 'Show Room - Corp. Sale-38'),
             ('area', 'Tanash'),
             ('Region', 'CAIRO'),
             ('adjusted_budget_gross_margin_percent', 10.0),
             ('budget_amount', 351045.4584868548),
             ('budget_cost', 315940.91263816936),
             ('budget_qty', 354.0),
             ('Chann

In [13]:
record_to_dict(records[0])

OrderedDict([('ITEMID', '1TSATVSM43T53000L007'),
             ('Description', 'TV SAMSUNG 43T5300 FHD SMART'),
             ('Department', 'DOMESTIC SAMSUNG'),
             ('Category', 'TV'),
             ('Family', 'TV'),
             ('SubFamily', 'TV DOM'),
             ('SubCategory', 'TV-43-SMRT'),
             ('ExtendedSubCategory', 'TV-43-SMRT'),
             ('SubCategorySupplier', 'TV-43-SMRT-SAMSUNG'),
             ('AssemblyCodeNickName',
              '1TSATVSM43T53000L007\nTV SAMSUNG 43T5300 FHD SMART'),
             ('ENDOFLife', None),
             ('DOM_COMM', 'DOM'),
             ('Status', 'No'),
             ('Supplier', 'SAMSUNG'),
             ('Store', 'Show Room - Corp. Sale-38'),
             ('area', 'Tanash'),
             ('Region', 'CAIRO'),
             ('adjusted_budget_gross_margin_percent', 10.0),
             ('budget_amount', 351045.4584868548),
             ('budget_cost', 315940.91263816936),
             ('budget_qty', 354.0),
             ('Chann

In [21]:
dictionary_of_records = [Rec_to_dict(record) for record in records]

In [22]:
# Convert the records to a Polars DataFrame
df = pl.from_records(dictionary_of_records)

In [23]:
df.head()

ITEMID,Description,Department,Category,Family,SubFamily,SubCategory,ExtendedSubCategory,SubCategorySupplier,AssemblyCodeNickName,ENDOFLife,DOM_COMM,Status,Supplier,Store,area,Region,adjusted_budget_gross_margin_percent,budget_amount,budget_cost,budget_qty,Channel,INVENTLOCATIONID,Budget_date,stock_on_hand_qty,opening_stock,closing_stock,current_stock_cost_at_retail,stock_received_qty,opening_stock_at_cost,adjustment_at_cost,adjustment_at_retail,stock_date,article_score_sale,article_score_abc,article_score_ae,article_score_speed,article_score_terminal,article_score_margin,article_score_sell,article_score_markdown,article_score_core,article_score_quartile,article_score_sortimeter,btech_vs_sortimeter,store_sku_count,price
str,str,str,str,str,str,str,str,str,str,null,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str,"decimal[*,0]","decimal[*,0]","decimal[*,0]",f64,i64,f64,i64,i64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64
"""1TSATVSM43T530…","""TV SAMSUNG 43T…","""DOMESTIC SAMSU…","""TV""","""TV""","""TV DOM""","""TV-43-SMRT""","""TV-43-SMRT""","""TV-43-SMRT-SAM…","""1TSATVSM43T530…",null,"""DOM""","""No""","""SAMSUNG""","""Show Room - Co…","""Tanash""","""CAIRO""",10.0,351045.458487,315940.912638,354.0,"""WAREHOUSE""","""Re-9011""","""2024-03-27""",8,8,8,0.0,0,0.0,0,0,2024-02-26,0.000061,0.000061,0.0,7.1986e-8,0.000082,-0.000195,0.000002,0.001116,0.000121,0.000005,0.0,0.0,0,0.0
"""1TSATVSM43T530…","""TV SAMSUNG 43T…","""DOMESTIC SAMSU…","""TV""","""TV""","""TV DOM""","""TV-43-SMRT""","""TV-43-SMRT""","""TV-43-SMRT-SAM…","""1TSATVSM43T530…",null,"""DOM""","""No""","""SAMSUNG""","""Show Room - Co…","""Tanash""","""CAIRO""",10.0,351045.458487,315940.912638,354.0,"""WAREHOUSE""","""Re-9011""","""2024-03-28""",8,8,8,0.0,0,0.0,0,0,2024-02-26,0.000061,0.000061,0.0,7.1986e-8,0.000082,-0.000195,0.000002,0.001116,0.000121,0.000005,0.0,0.0,0,0.0
"""1TLGTVSM43LM63…","""TV LG 43LM6370…","""DOMESTIC LG""","""TV""","""TV""","""TV DOM""","""TV-43-SMRT""","""TV-43-SMRT""","""TV-43-SMRT-LG""","""1TLGTVSM43LM63…",null,"""DOM""","""No""","""LG""","""Sohag Branch""","""unknown""","""unknown""",20.0,5076.164816,4060.931852,122.0,"""Retail""","""Re-4004""","""2024-03-27""",2,2,2,0.0,0,0.0,0,0,2024-02-26,0.000012,0.000012,0.0,7.1986e-8,0.000087,0.000164,0.000004,0.000114,0.000023,0.000034,0.0,0.0,0,0.0
"""1TLGTVSM43LM63…","""TV LG 43LM6370…","""DOMESTIC LG""","""TV""","""TV""","""TV DOM""","""TV-43-SMRT""","""TV-43-SMRT""","""TV-43-SMRT-LG""","""1TLGTVSM43LM63…",null,"""DOM""","""No""","""LG""","""Sohag Branch""","""unknown""","""unknown""",20.0,5076.164816,4060.931852,122.0,"""Retail""","""Re-4004""","""2024-03-28""",2,2,2,0.0,0,0.0,0,0,2024-02-26,0.000012,0.000012,0.0,7.1986e-8,0.000087,0.000164,0.000004,0.000114,0.000023,0.000034,0.0,0.0,0,0.0
"""1TLGTVSM43LM63…","""TV LG 43LM6370…","""DOMESTIC LG""","""TV""","""TV""","""TV DOM""","""TV-43-SMRT""","""TV-43-SMRT""","""TV-43-SMRT-LG""","""1TLGTVSM43LM63…",null,"""DOM""","""No""","""LG""","""Show Room - Co…","""Tanash""","""CAIRO""",20.0,500027.122173,400021.697738,156.0,"""WAREHOUSE""","""Re-9011""","""2024-03-27""",17,17,17,0.0,0,0.0,0,0,2024-02-26,0.000012,0.000012,0.0,7.1986e-8,0.000087,0.000164,0.000004,0.000114,0.000023,0.000034,0.0,0.0,0,0.0


In [6]:
import polars as pl

In [ ]:
{'Channel': ["B.TECH X","MINI", "WAREHOUSE","ARKAN","Retail", "NON","CALL CENTER"], 
 "Family":["TV","LAPTOP","SDA","MOBILE","MDA"],
 "SubFamily":["AC DOM", "MDA COM","SDA DOM","AC COM"],
 "ITEMID":[],"Category":[],
 "SubCategory":[],"Supplier":[]}